# (How to not yet) Introducing the Local Wavenumber Vector Concept to Sound Field Synthesis using Spherical and Circular Secondary Source Distributions

[Frank Schultz](https://orcid.org/0000-0002-3010-0294)

We want to show that high-frequency-/far-field-Approximated Near-field compensated (NFC) $\infty^\text{th}$-order Ambisonics == WFS in 2.5D SFS, analytically as best as possible.

See also our accompanying contribution:
Frank Schultz, Gergely Firtha, Fiete Winter, Sascha Spors (2019): "On the Connections of High-Frequency Approximated Ambisonics andWave Field Synthesis" In: Proc. of *45. Jahrestagung für Akustik (DAGA)*, Rostock.

## 0. Abstract / Motivation

In [Ch. 4.4.2, Ahrens Book](http://doi.org/10.1007/978-3-642-25743-8) it was stated that Wave Field Synthesis (WFS) constitutes a high-frequency approximation of Near-field Compensated Infinite Order Ambisonics (NFC-IOA). This was shown for a 2.5D scenario with a specific WFS type.

In [Firtha IEEE TASLP 2018](https://doi.org/10.1109/TASLP.2018.2865091) we have shown precise equivalence - using a linear array as secondary source distribution (SSD) - of 2.5D WFS and a high-frequency/far-field approximation of the so called Spectral Division Method (SDM), which constitutes the explicit sound field synthesis (SFS) solution in Cartesian coordinates.

In [Winter IEEE TASLP 2019](https://doi.org/10.1109/TASLP.2019.2892895) it is shown that direction of spatial aliasing grating lobes can be predicted by the local wavenumber vector concept.

This gives rise to the assumption that we can use this approach also for continuous SSDs in order to derive the following:

We want to show analytically that high-frequency approximated NFC-IOA (i.e. the explicit solution in circular/spherical coordinates) is precisely equivalent with the 2.5D unified/3D WFS solution with referencing scheme w.r.t. origin.

Although some calculus can be found in [Ch. 4.4.2, Ahrens Book](http://doi.org/10.1007/978-3-642-25743-8), this needs to be revised for proper driving function's referencing handling and for missing aspects, such as the impact of secondary source selection window required for WFS vs. the smooth inherent Ambisonics windowing. Thus, the approach of [Firtha IEEE TASLP 2018](https://doi.org/10.1109/TASLP.2018.2865091) - using the local wavenumber vector concept, cf. [Firtha's thesis](https://github.com/gfirtha/gfirtha_phd_thesis) - shall be adapted to spherical coordinates in hope for further insights to the links of WFS and HOA in 2.5D, potentially including pure panning approaches.

## 1. Tools and Conventions

### 1.1 sfs-python Toolbox 
We use the https://github.com/sfstoolbox/sfs-python toolbox for simulations embedded into this notebook.
See the file CONTRIBUTING.rst for install help.
Currently, the [fs446](https://github.com/sfstoolbox/sfs-python/tree/fs446) branch includes the latest code used by this notebook (latest check with SHA: c67db62).
Changes relate to new implementations and corrections of driving functions in wfs.py and nfchoa.py.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sfs
%matplotlib inline
matplotlib.rcParams['font.size'] = 8

### 1.2 Coordinate System Convention
We use radius $r$, azimuth angle $\phi$, polar angle $\theta$ for which $x = r \cos\phi \sin\theta$, $y = r \sin\phi \sin\theta$, $z = r\cos\theta$, and vector notation $\mathbf{x} = (x,y,z)^\mathrm{T}$ holds. Vector magnitude is denoted as $|\cdot|$ and inner vector product is written as $\langle \cdot,\cdot \rangle$.

### 1.3 Time Convention 
We use the $\mathrm{e}^{+ \mathrm{j} \omega t}$-time convention for monochromatic sound fields. Speed of sound is denoted by $c$, assuming constant 343 m/s here. The temporal angular frequency $\omega$ is linked to the frequency $f$ by $\omega=2 \pi f$.

### 1.4 Point Source
The sound field of a radiating point source at position $\mathbf{x}_S$ is
$P(\mathbf{x},\omega) = \frac{1}{4\pi} \frac{\mathrm{e}^{- \mathrm{j} \frac{\omega}{c} |\mathbf{x}-\mathbf{x}_{S}|}}{|\mathbf{x}-\mathbf{x}_{S}|} \mathrm{e}^{+ \mathrm{j} \omega t}$.
Due to $\frac{1}{4 \pi}$-normalization this also constitutes the 3D freefield Green's function. For derivation of point source driving functions we use this field $P(\mathbf{x},\omega)$ as the primary/target/virtual sound field.

### 1.5 Plane Wave

The sound field of a propagating plane wave into direction of unit vector $\mathbf{\hat{k}}_{PW}$ and unit amplitude is
$P(\mathbf{x},\omega) = \mathrm{e}^{- \mathrm{j} \frac{\omega}{c}\langle \mathbf{\hat{k}}_{PW}, \mathbf{x} \rangle} \mathrm{e}^{+ \mathrm{j} \omega t}$.

### 1.6 Secondary Source Distribution
The secondary source distribution (SSD) is a circle positioned around origin consisting of 3D freefield Green's functions. Secondary source positions are denoted by $\mathbf{x}_0$, such that radius $r_0 = |\mathbf{x}_0|$. The inward unit normal at position $\mathbf{x}_0$ is given as $\mathbf{\hat{n}}_0(\mathbf{x}_0) = - \frac{\mathbf{x}_0}{r_0}$. In the remainder we use the $xy$-plane as SFS plane, thus $z$-components are zero in all vectors, i.e. $\theta=\frac{\pi}{2}$. The circular SSD in $xy$-plane is thus described by $x_0=r_0\cos\phi_0$, $y_0=r_0\sin\phi_0$, $z_0=0$. We use the origin as the referencing point, i.e. $\mathbf{x}_{Ref}=(0,0,0)^\mathrm{T}$.

### 1.7 Synthesis Integral

The single layer potential for 2.5D SFS of listening space $r<r_0$ using a circular SSD with radius $R=r_0$ reads

$S(\mathbf{x},\omega) = \int_{0}^{2\pi} D(\mathbf{x}_0,\omega) \frac{1}{4\pi} \frac{\mathrm{e}^{- \mathrm{j} \frac{\omega}{c} |\mathbf{x}-\mathbf{x}_{0}|}}{|\mathbf{x}-\mathbf{x}_{0}|} R \, \mathrm{d}\phi_0 \, \mathrm{e}^{+ \mathrm{j} \omega t}$,

for which different driving functions $D(\mathbf{x}_0,\omega)$ can be utilized.

### 1.8 Simulation Setup NFC-HOA vs WFS
Next, we prepare a plot routine for the upcoming simulations.

In [ ]:
def plot_routine(file):
    phi0 = np.arange(0, 360, 360 / L)
    fig_width = 12
    fig_ratio = 8 / 16
    plt.figure(figsize=(fig_width, fig_ratio * fig_width))

    plt.subplot(241)
    sfs.plot.soundfield(p_HOA, grid / wavelength,
                        xnorm=None, cmap="RdBu_r",
                        vmin=-2e-0, vmax=+2e-0,
                        xlabel=r"$x/\lambda$", ylabel=r"$y/\lambda$")
    sfs.plot.loudspeaker_2d(x0 / wavelength, n0, tapering_window * 0 + 1)
    print("check that", kr0, "<", M, "to avoid spatial aliasing")
    if M < kr0:  # plot circle of aliasing free region
        phi = np.arange(0, 2 * np.pi, 2 * np.pi / (2 ** 6))
        plt.plot(M / k * np.cos(phi) / wavelength,
                 M / k * np.sin(phi) / wavelength,
                 color=(0.5, 0.5, 0.5))
    plt.grid()
    plt.title("HOA");

    plt.subplot(242)
    sfs.plot.soundfield(p_WFS, grid / wavelength,
                        xnorm=None, cmap="RdBu_r",
                        vmin=-2e-0, vmax=+2e-0,
                        xlabel=r"$x/\lambda$", ylabel=r"$y/\lambda$")
    sfs.plot.loudspeaker_2d(x0 / wavelength, n0, tapering_window)
    plt.grid()
    plt.title("WFS");

    plt.subplot(243)
    sfs.plot.soundfield((p_WFS - p_HOA) / p_HOA, grid / wavelength,
                        xnorm=None, cmap="seismic",
                        vmin=-1e-1, vmax=+1e-1,
                        xlabel=r"$x/\lambda$", ylabel=r"$y/\lambda$")
    sfs.plot.loudspeaker_2d(x0 / wavelength, n0, tapering_window * 0)
    plt.grid()
    plt.title("(WFS-HOA)/HOA");

    plt.subplot(244)
    sfs.plot.soundfield(20 * np.log10(np.abs((p_WFS - p_HOA) / p_HOA)),
                        grid / wavelength,
                        xnorm=None, cmap="inferno_r",
                        vmin=-60, vmax=+0,
                        xlabel=r"$x/\lambda$", ylabel=r"$y/\lambda$")
    sfs.plot.loudspeaker_2d(x0 / wavelength, n0, tapering_window * 0)
    plt.grid()
    plt.title("|(WFS-HOA)/HOA| in dB");

    plt.subplot(245)
    sfs.plot.soundfield(20 * np.log10(np.abs(p_HOA)), grid / wavelength,
                        xnorm=None, cmap="seismic",
                        vmin=-1e+1, vmax=+1e+1,
                        xlabel=r"$x/\lambda$", ylabel=r"$y/\lambda$")
    sfs.plot.loudspeaker_2d(x0 / wavelength, n0, tapering_window * 0 + 1)
    plt.grid()
    plt.title("|HOA| in dB");

    plt.subplot(246)
    sfs.plot.soundfield(20 * np.log10(np.abs(p_WFS)), grid / wavelength,
                        xnorm=None, cmap="seismic",
                        vmin=-1e1, vmax=+1e1,
                        xlabel=r"$x/\lambda$", ylabel=r"$y/\lambda$")
    sfs.plot.loudspeaker_2d(x0 / wavelength, n0, tapering_window)
    plt.grid()
    plt.title("|WFS| in dB");

    plt.subplot(247)
    sfs.plot.soundfield(
        20 * np.log10(np.abs(p_HOA)) - 20 * np.log10(np.abs(p_WFS)),
        grid / wavelength,
        xnorm=None, cmap="seismic",
        vmin=-2e-1, vmax=+2e-1,
        xlabel=r"$x/\lambda$", ylabel=r"$y/\lambda$")
    sfs.plot.loudspeaker_2d(x0 / wavelength, n0, tapering_window * 0)
    plt.grid()
    plt.title("|WFS| in dB - |HOA| in dB");

    plt.subplot(248)
    sfs.plot.soundfield(
        np.unwrap(np.angle(p_HOA)) - np.unwrap(np.angle(p_WFS)),
        grid / wavelength,
        xnorm=None, cmap="seismic",
        vmin=-np.pi / 4, vmax=+np.pi / 4,
        xlabel=r"$x/\lambda$", ylabel=r"$y/\lambda$")
    sfs.plot.loudspeaker_2d(x0 / wavelength, n0, tapering_window * 0)
    plt.grid()
    plt.title("arg(WFS) - arg(HOA)");

    plt.savefig(file + '_soundfield.png')
    
    
    plt.figure(figsize=(fig_width, fig_ratio * fig_width))
    plt.subplot(211)
    plt.plot(phi0, 20 * np.log10(np.abs(d_HOA * a0)), "C3", label="|HOA|")
    plt.plot(phi0, 20 * np.log10(np.abs(d_WFS * tapering_window * a0)), "C0",
             label="|WFS|")
    plt.plot(phi0,
             20 * np.log10(np.abs(d_HOA * a0) / np.abs(d_WFS * tapering_window * a0)),
             "C9", label="|HOA|/|WFS|")
    plt.plot(phi0, 20 * np.log10(
        np.abs(((d_WFS * tapering_window * a0) - (d_HOA * a0)) / (d_HOA * a0))), "C8",
             label="|(WFS-HOA)/HOA|")
    plt.xlim(0, 360)
    plt.legend()
    plt.grid()
    plt.xlabel("$\phi_0$ in deg")
    plt.ylabel("magnitude in dB")
    plt.title("driving function")
    plt.subplot(212)
    plt.plot(phi0, np.angle(d_HOA * a0), "C3", label="HOA")
    plt.plot(phi0, np.angle(d_WFS * tapering_window * a0), "C0", label="WFS")
    plt.plot(phi0, np.angle(d_HOA * a0) - np.angle(d_WFS * tapering_window * a0),
             "C8", label="$\phi$(HOA)-$\phi$(WFS)")
    plt.xlim(0, 360)
    plt.legend()
    plt.grid()
    plt.xlabel("$\phi_0$ in deg")
    plt.ylabel("phase in rad")

    plt.savefig(file + '_drivingfilter.png')    
    
    print("HOA Mag at P(0,0):", np.abs(sfs.util.probe(p_HOA, grid, [0, 0, 0])))
    print("WFS Mag at P(0,0):", np.abs(sfs.util.probe(p_WFS, grid, [0, 0, 0])))

    print("HOA Ph at P(0,0):",
          np.angle(sfs.util.probe(p_HOA, grid, [0, 0, 0])))
    print("WFS Ph at P(0,0):",
          np.angle(sfs.util.probe(p_WFS, grid, [0, 0, 0])))

    print("f =", f, "Hz, wavelength = ", wavelength, "m, r0 =", r0, "m")
    print("kr0 =", 2 * np.pi / wavelength * r0)  # kr0 >> 1

## 2. Driving Functions

### 2.1 General 2.5D WFS Driving Function (cf. [Firtha IEEE TASLP 2017](https://doi.org/10.1109/TASLP.2017.2689245), [Firtha IEEE TASLP 2018](https://doi.org/10.1109/TASLP.2018.2865091), [Winter IEEE TASLP 2019](https://doi.org/10.1109/TASLP.2019.2892895))
The driving function according to unified 2.5D Wave Field Synthesis (WFS) theory reads

$D(\mathbf{x}_0,\omega) = w(\mathbf{x}_0) \sqrt{\frac{\mathrm{j} \omega}{c}} \sqrt{8 \pi} \sqrt{d(\mathbf{x}_0)}
\langle \mathbf{\hat{k}}_S(\mathbf{x}_0), \mathbf{\hat{n}}_0(\mathbf{x}_0)\rangle S(\mathbf{x}_0,\omega)$

with the referencing function $d(\mathbf{x}_0)$ (note the difference between $d(\mathbf{x}_0)$ and $D(\mathbf{x}_0)$) and the local wavenumber vector $\mathbf{\hat{k}}_S(\mathbf{x}_0)$ of the sound field $S(\mathbf{x}_0)$ at position $\mathbf{x}_0$. Inherent to all 2.5D WFS driving functions is the $\sqrt{\mathrm{j} \omega}$ prefilter with 3 dB per octave highpass characteristics and +45 deg phase shift for all frequencies, cf. Ch. 2.5 in [Schultz Thesis 2016](https://doi.org/10.18453/rosdok_id00001765).

### 2.2 Point Source 2.5D WFS Driving Function (cf. [Schultz Thesis 2016](https://doi.org/10.18453/rosdok_id00001765), [Winter IEEE TASLP 2019](https://doi.org/10.1109/TASLP.2019.2892895))
Using the normalized local wavenumber vector $\mathbf{\hat{k}}_S(\mathbf{x}_0) = \frac{\mathbf{x}_0-\mathbf{x}_{S}}{|\mathbf{x}_0-\mathbf{x}_{S}|}$ of the point source and the referencing function
$d(\mathbf{x}_0) = \frac{|\mathbf{x}_0-\mathbf{x}_{S}| \cdot |\mathbf{x}_{Ref}-\mathbf{x}_{0}|}{|\mathbf{x}_0-\mathbf{x}_{S}| + |\mathbf{x}_{Ref}-\mathbf{x}_{0}|}$,
as well as for desired field
$S(\mathbf{x},\omega) == P(\mathbf{x},\omega)$ deploying $S(\mathbf{x}_0,\omega)$ on the SSD,
yields the 2.5D WFS driving function for a point source with referencing to position $\mathbf{x}_{Ref}$

$D(\mathbf{x}_0,\omega) = w(\mathbf{x}_0) \sqrt{\frac{\mathrm{j \omega}}{c}} \sqrt{8 \pi}
\sqrt{\frac{|\mathbf{x}_0-\mathbf{x}_{S}| \cdot |\mathbf{x}_{Ref}-\mathbf{x}_{0}|}{|\mathbf{x}_0-\mathbf{x}_{S}| + |\mathbf{x}_{Ref}-\mathbf{x}_{0}|}}
\frac{\langle \mathbf{x}_0-\mathbf{x}_{S}, \mathbf{\hat{n}}_0(\mathbf{x}_0)\rangle}{|\mathbf{x}_0-\mathbf{x}_{S}|}
\frac{1}{4\pi} \frac{\mathrm{e}^{- \mathrm{j} \frac{\omega}{c} |\mathbf{x}_0-\mathbf{x}_{S}|}}{|\mathbf{x}_0-\mathbf{x}_{S}|}$,
cf. (2.137) in the cited thesis, (8) in [Winter IEEE TASLP 2019](https://doi.org/10.1109/TASLP.2019.2892895). 

The secondary source selection (spatial window) is defined as
$w(\mathbf{x}_0) \qquad = 1\,\,\text{if} \langle \mathbf{\hat{k}}_S(\mathbf{x}_0),\mathbf{\hat{n}}_0(\mathbf{x}_0)\rangle>0, \qquad 0\,\,\text{otherwise}$.

This driving function is implemented in function [wfs_25d_point_Unified_WIP()](https://github.com/sfstoolbox/sfs-python/blob/fs446/sfs/mono/drivingfunction.py) currently in the branch 'fs446' using the variables
$ds = \mathbf{x}_0-\mathbf{x}_{S}$, $dr = \mathbf{x}_{Ref}-\mathbf{x}_{0}$,
$s = |\mathbf{x}_0-\mathbf{x}_{S}|$, $r = |\mathbf{x}_{Ref}-\mathbf{x}_{0}|$
for sake of brevity. It then reads

$D(\mathbf{x}_0,\omega) = w(\mathbf{x}_0) \sqrt{\frac{\mathrm{j \omega}}{c}} \sqrt{8 \pi}
\sqrt{\frac{r \cdot s}{r + s}}
\frac{\langle ds, \mathbf{\hat{n}}_0(\mathbf{x}_0)\rangle}{s}
\frac{1}{4\pi} \frac{\mathrm{e}^{- \mathrm{j} \frac{\omega}{c} s}}{s}$.

### 2.3 Point Source  2.5D NFC-HOA Driving Function (cf. [Ahrens Book](http://doi.org/10.1007/978-3-642-25743-8))
The 2.5D nearfield-compensated higher-order Ambisonics (NFC-HOA) driving function (cf. eq. (5.8) in the cited book) reads

$D(\mathbf{x}_0,\omega) = \frac{1}{2 \pi r_0} \sum\limits_{m=-M}^{+M} \frac{h^{(2)}_{|m|}(\frac{\omega}{c} r_S)}{h^{(2)}_{|m|}(\frac{\omega}{c} r_0)} \mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_S)}$

using [spherical Hankel function](https://dlmf.nist.gov/10.47#E1) $h_\nu^{(2)}(\cdot)$ of second kind of order $\nu$. Summation up to modal (integer) order $M\leq \frac{L}{2}$ for SSD of $L$ secondary sources is proposed to avoid spatial aliasing. In the sfs-python toolbox this function is implemented in function [nfchoa_25d_point()](https://github.com/sfstoolbox/sfs-python/blob/fs446/sfs/mono/drivingfunction.py).

### 2.4 Plane Wave 2.5D WFS Driving Function (cf. [Schultz Thesis 2016](https://doi.org/10.18453/rosdok_id00001765))

The 2.5D unified WFS driving function for a plane wave referencing to point $\mathbf{x}_{Ref}$ is given as

$D(\mathbf{x}_0,\omega) = w(\mathbf{x}_0) \sqrt{\frac{\mathrm{j \omega}}{c}} \sqrt{8 \pi} \cdot
\sqrt{|\mathbf{x}_{Ref}-\mathbf{x}_{0}|} \cdot
\langle \mathbf{\hat{k}}_{PW}, \mathbf{\hat{n}}(\mathbf{x}_0) \rangle \cdot
\mathrm{e}^{- \mathrm{j} \frac{\omega}{c}\langle \mathbf{\hat{k}}_{PW}, \mathbf{x}_0 \rangle}$,

cf. (2.177) in the cited thesis. The referencing function is given as $d(\mathbf{x}_0) = |\mathbf{x}_{Ref}-\mathbf{x}_{0}|$, cf. [Firtha IEEE TASLP 2017](https://doi.org/10.1109/TASLP.2017.2689245).

The secondary source selection (spatial window) is defined as
$w(\mathbf{x}_0) = 1\quad \text{if} \langle \mathbf{\hat{k}}_{PW}(\mathbf{x}_0),\mathbf{\hat{n}}_0(\mathbf{x}_0)\rangle>0,\,\,0 \quad \text{otherwise}$.

### 2.5 Plane Wave 2.5D NFC-HOA Driving Function (cf. [Ahrens Book](http://doi.org/10.1007/978-3-642-25743-8))

The 2.5D NFC-HOA driving function for a plane wave reads (cf. eq. (5.1) in the cited book, corrected here)

$D(\mathbf{x}_0,\omega) = \sum\limits_{m=-M}^{+M} \frac{2 \mathrm{j}}{\frac{\omega}{c} r_0} \frac{(-\mathrm{j})^{|m|}}{h_{|m|}^{(2)}(\frac{\omega}{c} r_0)} \mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_{PW})}$.

## 2.6 Comparison NFC-HOA vs. WFS -> Simulation

We define a setup for which far-field/high frequency approximation is valid. We use normalized $kr$ resp. $x/\lambda$-domain rather than absolute values, since the characteristics appear proportional to $kr$ with appropriate chosen modal bandwidth.

In [ ]:
wavelength = 1  # in m
c = 343  # speed of sound in m/s

far = 10
kr0 = far*np.pi  # very large for valid far/hf approx
krs = (far+far)*np.pi  # very large for valid far/hf approx

k = 2*np.pi/wavelength  # in rad/m
f =  c/wavelength  # frequency in Hz
omega = 2*np.pi*f  # rad/s
r0 = kr0/k  # radius of spherical/circular array in m
rs = krs/k  # distance of primary source from origin
M = int(np.ceil(krs))  # number of modes, note the chosen dependency of rs rather than r0
L = M*2  # number of secondary sources, >M*2 to avoid spatial aliasing
grid_spacing = wavelength/16

xref = [0, 0, 0]  # reference position for 2.5D SFS
grid = sfs.util.xyz_grid([-np.ceil(r0), +np.ceil(r0)], [-np.ceil(r0), +np.ceil(r0)], 0, spacing = grid_spacing)
ssd = sfs.array.circular(L, r0)
x0 = ssd.x
n0 = ssd.n
a0 = ssd.a
print(wavelength, f, k, r0, rs, M, L, kr0, krs)

### 2.6.1 Spherical Wave / Point Source NFC-HOA vs. WFS

In [ ]:
#SPHERICAL WAVE
phis = 4*np.pi/4  # angle of primary source position 
xs = [rs*np.cos(phis), rs*np.sin(phis), 0]  # position of primary source
A = np.linalg.norm(xs)*4*np.pi  # amplitude point source, i.e. amplitude 1 at origin
print("normalized distance PS to SSD =", (np.linalg.norm(xs)-r0)/wavelength, "wavelengths")

print(xs)

d_HOA, sssel, ssfunc = sfs.mono.nfchoa.point_25d(omega, ssd.x, r0, xs)
tapering_window = sfs.tapering.kaiser(sssel, 0)
p_HOA = sfs.mono.synthesize(A*d_HOA, tapering_window, ssd, ssfunc, grid=grid)

d_WFS, sssel, ssfunc = sfs.mono.wfs.point_25d_Unified_WIP(omega, ssd.x, ssd.n, xs)
tapering_window = sfs.tapering.kaiser(sssel, 0)
p_WFS = sfs.mono.synthesize(A*d_WFS, tapering_window, ssd, ssfunc, grid=grid)

plot_routine('Point Source WFS vs HOA')

### 2.6.2 Plane Wave NFC-HOA vs. WFS

In [ ]:
#PLANE WAVE
pw_angle = 0  # propagating direction of plane wave in deg
npw = sfs.util.direction_vector(np.radians(pw_angle), np.radians(90))
A = 1 # i.e. amplitude 1 at origin

d_HOA, sssel, ssfunc = sfs.mono.nfchoa.plane_25d(omega, x0, r0, npw)
tapering_window = sfs.tapering.kaiser(sssel, 0)
p_HOA = sfs.mono.synthesize(A*d_HOA, tapering_window, ssd, ssfunc, grid=grid)

d_WFS, sssel, ssfunc = sfs.mono.wfs.plane_25d(omega, x0, n0, npw, xref)
tapering_window = sfs.tapering.kaiser(sssel, 0)
p_WFS = sfs.mono.synthesize(A*d_WFS, tapering_window, ssd, ssfunc, grid=grid)

plot_routine('Plane Wave WFS vs HOA')

In [ ]:
def plane_25d_exactref2origin(omega, x0, n0, n, c=None):
    # under development with G. Firtha:
    x0 = sfs.util.asarray_of_rows(x0)
    n0 = sfs.util.asarray_of_rows(n0)
    n = sfs.util.normalize_vector(n)

    k = sfs.util.wavenumber(omega, c)
    phi0 = np.arctan2(x0[:,1], x0[:,0])
    phiPW = np.arctan2(n[1], n[0])
    r0 = np.linalg.norm(x0[0,:]) 
    phi0[phi0<0] = phi0[phi0<0] + 2*np.pi
    
    d = -np.sqrt(-8*np.pi*1j*k*r0) *\
    np.exp(1j*(phi0-phiPW)/2) *\
    np.cos(phi0-phiPW) *\
    np.exp(-1j*(r0*k*np.cos(phi0-phiPW)))
    selection = sfs.util.source_selection_plane(n0, n)
    return d, selection, sfs.mono.secondary_source_point(omega, c)

In [ ]:
#PLANE WAVE
pw_angle = 0  # propagating direction of plane wave in deg
npw = sfs.util.direction_vector(np.radians(pw_angle), np.radians(90))
A = 1 # i.e. amplitude 1 at origin

d_HOA, sssel, ssfunc = sfs.mono.nfchoa.plane_25d(omega, x0, r0, npw)
tapering_window = sfs.tapering.kaiser(sssel, 0)
p_HOA = sfs.mono.synthesize(A*d_HOA, tapering_window, ssd, ssfunc, grid=grid)

d_WFS, sssel, ssfunc = plane_25d_exactref2origin(omega, x0, n0, npw)
tapering_window = sfs.tapering.kaiser(sssel, 0)
p_WFS = sfs.mono.synthesize(A*d_WFS, tapering_window, ssd, ssfunc, grid=grid)

plot_routine('Plane Wave WFS vs HOA')

In [ ]:
# DAGA 2019 Plot
#matplotlib.rcParams['font.size'] = 10
#phi0 = np.arange(0, 360, 360 / L)
#fig_width = 6
#fig_height = 3

#plt.figure(figsize=(fig_width, fig_height))

#plt.subplot(121)
#sfs.plot.soundfield(20 * np.log10(np.abs(p_HOA)), grid / wavelength,
#                    xnorm=None, cmap="seismic",
#                    vmin=-1e+1, vmax=+1e+1,
#                    xlabel=r"$x / \lambda$", ylabel=r"$y / \lambda$",
#                   colorbar=False)
#sfs.plot.loudspeaker_2d(x0 / wavelength, n0, tapering_window * 0 + 1)
#plt.grid()
#plt.title("|HOA| in dB");

#plt.subplot(122)
#sfs.plot.soundfield(20 * np.log10(np.abs(p_WFS)), grid / wavelength,
#                    xnorm=None, cmap="seismic",
#                    vmin=-1e1, vmax=+1e1,
#                    xlabel=r"$x / \lambda$", ylabel="",
#                   colorbar=True)
#sfs.plot.loudspeaker_2d(x0 / wavelength, n0, tapering_window)
#plt.grid()
#plt.title("|WFS| in dB");
#plt.savefig('HOA_WFS_RefLine_kr'+str(kr0))

## 3. FarField/HF NFC-IOA vs. WFS

### 3.1 Spherical Wave / Point Source
FarField/HF NFC-IOA vs. WFS

The spherical Hankel functions can be approximated for very large arguments as [eq. (6.68) in Williams Fourier](https://doi.org/10.1016/B978-0-12-753960-7.X5000-1)
$h_\nu^{(2)}(x) \approx \mathrm{j}^{\nu+1} \frac{\mathrm{e}^{-\mathrm{j} x}}{x}$.
Considering this for the driving function above yields

$D(\mathbf{x}_0,\omega)
= \frac{1}{2 \pi r_0} \sum \limits_{m=-M}^{+M} \frac{h^{(2)}_{|m|}(\frac{\omega}{c} r_S)}{h^{(2)}_{|m|}(\frac{\omega}{c} r_0)} \mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_S)}
\approx
\frac{1}{2 \pi r_0} \sum\limits_{m=-M}^{+M}
\frac
{\mathrm{j}^{|m|+1} \frac{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_S}}{\frac{\omega}{c} r_S}}
{\mathrm{j}^{|m|+1} \frac{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}{\frac{\omega}{c} r_0}}
\mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_S)}
=
\frac{1}{2 \pi r_0} \sum\limits_{m=-M}^{+M}
\frac
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_S}}
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\frac{r_0}{r_S}
\mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_S)}$.

We move terms out of the sum, towards

$D(\mathbf{x}_0,\omega) =
\frac{1}{2 \pi r_0}
\frac
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_S}}
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\frac{r_0}{r_S}
\sum\limits_{m=-M}^{+M}
\mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_S)}$
and identify the sum of exponentials as [Dirichlet kernel](https://en.wikipedia.org/wiki/Dirichlet_kernel). For $M\rightarrow \infty$ (i.e. using infinite order Ambisonics as intended in the first instance) this kernel merges to the Dirac distribution and thus

$D_\text{HF}(\mathbf{x}_0,\omega) =
\frac{1}{r_0}
\frac
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_S}}
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\frac{r_0}{r_S}
\delta(\phi_0 - \phi_S)$.

This driving function has interesting implications that precisely correspond to the findings in [Ch. 4 Firtha thesis](https://github.com/gfirtha/gfirtha_phd_thesis), [Firtha IEEE TASLP 2017](https://doi.org/10.1109/TASLP.2017.2689245):

* only one secondary source is active, i.e. precisely the one for which $\phi_0 =  \phi_S$ holds. This is the stationary phase point denoted here with $\mathbf{x}_0^* = (r_0 \sin(\phi_S), r_0 \cos(\phi_S), 0)^\mathrm{T}$.

* the amplitude correction factor matches the amplitude of the intended virtual source in the origin, i.e. compensating amplitude decay of secondary source by $r_0$ and instead applying decay of virtual source by $\frac{1}{r_S}$. Note that the first term $\frac{1}{r_0}$ cancels out due to the synthesis integral, shown later.

* same correction is applied for the phase in the origin, i.e. compensation of the secondary source propagating time by $\mathrm{e}^{+\mathrm{j} \frac{\omega}{c} r_0}$ and instead applying $\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_S}$ as delay for the intended virtual source.

This becomes more obvious if inserting this driving function into the synthesis integral (i.e. single layer potential)

$P(\mathbf{x},\omega) = \int\limits_{\phi_0=0}^{2 \pi} D_\text{HF}(\mathbf{x}_0,\omega) \frac{1}{4\pi} \frac{\mathrm{e}^{- \mathrm{j} \frac{\omega}{c} |\mathbf{x}-\mathbf{x}_{0}|}}{|\mathbf{x}-\mathbf{x}_{0}|} r_0 \mathrm{d}\phi_0$,
which yields

$P(\mathbf{x},\omega) = \int\limits_{\phi_0=0}^{2 \pi} \frac{1}{r_0}
\frac
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_S}}
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\frac{r_0}{r_S}
\delta(\phi_0 - \phi_S) \frac{1}{4\pi} \frac{\mathrm{e}^{- \mathrm{j} \frac{\omega}{c} |\mathbf{x}-\mathbf{x}_{0}|}}{|\mathbf{x}-\mathbf{x}_{0}|} r_0 \mathrm{d}\phi_0=
\frac
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_S}}
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\frac{r_0}{r_S}
\frac{1}{4\pi} \frac{\mathrm{e}^{- \mathrm{j} \frac{\omega}{c} |\mathbf{x}-\mathbf{x}_{0}^*|}}{|\mathbf{x}-\mathbf{x}_{0}^*|}$.

In the origin $|\mathbf{x}-\mathbf{x}_{0}^*|=r_0$ holds and thus
$P(\mathbf{0},\omega) = 
\frac
{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_S}}
{4 \pi r_S}$
remains as the expected value for the pressure there.

We see:

A single secondary source at position $\mathbf{x}_{0}^*$ (i.e. the source of stationary phase) produces a spherical wave front that matches for the origin's position in magnitude and phase with the intended virtual point source at position $\mathbf{x}_{S}$. At all other positions amplitude and potentially phase errors will occur, since the wavefront of a single secondary source is not able to reproduce the intended wavefront of the virtual source (in the example here almost a planar wavefront within the SSD due to a very distant point source). Only, along the line $\mathbf{x}_S$ to the origin $\mathbf{0}$ (and further on that line) the phase of the wavefront is correct as well, however exhibiting the typical 2.5D SFS amplitude errors.

The simulation below demonstrates this behaviour with the high-frequency approximated NFC Infinite Order Ambisonics driving function implemented in the [nfchoa_25d_point_HF()](https://github.com/sfstoolbox/sfs-python/blob/fs446/sfs/mono/drivingfunction.py) function.

In [ ]:
#SPHERICAL WAVE  HF Approx
phis = 4*np.pi/4  # angle of primary source position 
xs = [rs*np.cos(phis), rs*np.sin(phis), 0]  # position of primary source
A = np.linalg.norm(xs)*4*np.pi  # amplitude point source, i.e. amplitude 1 at origin
print("normalized distance PS to SSD =", (np.linalg.norm(xs)-r0)/wavelength, "wavelengths")

d_HOA, sssel, ssfunc = sfs.mono.nfchoa.point_25d_HF(omega, x0, r0, xs,DirichletKernelFlag=False)
tapering_window = sfs.tapering.kaiser(sssel, 0)
p_HOA = sfs.mono.synthesize(A*d_HOA, tapering_window, ssd, ssfunc, grid=grid)

d_WFS, sssel, ssfunc = sfs.mono.wfs.point_25d_Unified_WIP(omega, ssd.x, ssd.n, xs)
tapering_window = sfs.tapering.kaiser(sssel, 0)
p_WFS = sfs.mono.synthesize(A*d_WFS, tapering_window, ssd, ssfunc, grid=grid)

plot_routine('Point Source HF WFS vs HOA')

### 3.2 Plane Wave
FarField/HF NFC-IOA vs. WFS

The same calculus using $h_\nu^{(2)}(x) \approx \mathrm{j}^{\nu+1} \frac{\mathrm{e}^{-\mathrm{j} x}}{x}$ [eq. (6.68)](https://doi.org/10.1016/B978-0-12-753960-7.X5000-1) 
is performed for the NFC-HOA plane wave driving function

$D(\mathbf{x}_0,\omega) = \sum\limits_{m=-M}^{+M} \frac{2 \mathrm{j}}{\frac{\omega}{c} r_0} \frac{(-\mathrm{j})^{|m|}}{h_{|m|}^{(2)}(\frac{\omega}{c} r_0)} \mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_{PW})}$

yielding

$D(\mathbf{x}_0,\omega) = \sum\limits_{m=-M}^{+M} \frac{2 \mathrm{j}}{\frac{\omega}{c} r_0} \frac{(-\mathrm{j})^{|m|}}{\mathrm{j}^{|m|+1} \frac{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}{\frac{\omega}{c} r_0}} \mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_{PW})}$.

Reordering yields

$D(\mathbf{x}_0,\omega) =
\frac{2}{\frac{\omega}{c} r_0} \frac{\frac{\omega}{c} r_0}{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\sum\limits_{m=-M}^{+M} \frac{\mathrm{j} (-\mathrm{j})^{|m|}}{\mathrm{j}^{|m|+1}} \mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_{PW})}
=
\frac{2}{r_0} \frac{r_0}{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\sum\limits_{m=-M}^{+M} \mathrm{e}^{\mathrm{j} \pi m} \, \mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_{PW})}$.

We might simplify this by introducing the angle $\phi_{PWi} = \phi_{PW} - \pi$ for plane wave incidence rather than using the plane wave propagating direction $\phi_{PW}$. This results in

$D(\mathbf{x}_0,\omega)=
\frac{2}{r_0} \frac{r_0}{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\sum\limits_{m=-M}^{+M} \mathrm{e}^{\mathrm{j} \pi m} \, \mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_{PWi} - \pi)}
=
\frac{2}{r_0} \frac{r_0}{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\sum\limits_{m=-M}^{+M} \, \mathrm{e}^{\mathrm{j} m (\phi_0 - \phi_{PWi})}$

obtaining the Dirichlet kernel again. For $M\rightarrow \infty$ the kernel merges to the Dirac distribution

$D(\mathbf{x}_0,\omega)=
2 \pi \frac{2}{r_0} \frac{r_0}{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\delta(\phi_0 - \phi_{PWi})
=
\frac{1}{r_0} \frac{4 \pi r_0}{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\delta(\phi_0 - \phi_{PWi})$.

We see again, that the driving function 
* activates only one single secondary source $\mathbf{x}_0^*$, i.e. of the stationary phase where $\phi_0 = \phi_{PWi}$
* compensates the amplitude decay of this source towards the plane wave amplitude in the origin, i.e. 1
* compensates the phase shift of this source towards the phase shift of the plane wave in the origin, i.e. 0

Introducing the driving function into the synthesis integral it becomes more obvious

$P(\mathbf{x},\omega) = \int\limits_{\phi_0=0}^{2 \pi} 
\frac{1}{r_0} \frac{4 \pi r_0}{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\delta(\phi_0 - \phi_{PWi})
\frac{1}{4\pi} \frac{\mathrm{e}^{- \mathrm{j} \frac{\omega}{c} |\mathbf{x}-\mathbf{x}_{0}|}}{|\mathbf{x}-\mathbf{x}_{0}|} r_0 \mathrm{d}\phi_0
=
\frac{r_0}{\mathrm{e}^{-\mathrm{j} \frac{\omega}{c} r_0}}
\frac{\mathrm{e}^{- \mathrm{j} \frac{\omega}{c} |\mathbf{x}-\mathbf{x}_{0}^*|}}{|\mathbf{x}-\mathbf{x}_{0}^*|}$

In the origin $|\mathbf{x}-\mathbf{x}_{0}^*|=r_0$ holds and thus
$P(\mathbf{0},\omega) = 1
$
remains as the expected value for the pressure there.

In [ ]:
#PLANE WAVE HF Approx
pw_angle = 0  # propagating direction of plane wave in deg
npw = sfs.util.direction_vector(np.radians(pw_angle), np.radians(90))
A = 1 # i.e. amplitude 1 at origin

d_HOA, sssel, ssfunc = sfs.mono.nfchoa.plane_25d_HF(omega, x0, r0, npw, DirichletKernelFlag=False)
tapering_window = sfs.tapering.kaiser(sssel, 0)
p_HOA = sfs.mono.synthesize(A*d_HOA, tapering_window, ssd, ssfunc, grid=grid)

d_WFS, sssel, ssfunc = sfs.mono.wfs.plane_25d(omega, x0, n0, npw, xref)
tapering_window = sfs.tapering.kaiser(sssel, 0)
p_WFS = sfs.mono.synthesize(A*d_WFS, tapering_window, ssd, ssfunc, grid=grid)

plot_routine('Plane Wave HF WFS vs HOA')

## 4 Circular Harmonics Expansion of Plane Wave 2.5D WFS Driving Function

We check [Ch. 4.4.2, Ahrens Book](http://doi.org/10.1007/978-3-642-25743-8) here in detail. Very helpful treatment on driving function expansions can be found in [Nara Hahn AES140th](http://www.aes.org/e-lib/browse.cfm?elib=18294).

The 2.5D unified WFS driving function for a plane wave referencing to point $\mathbf{x}_{Ref}$ was above given as, cf. [(2.177)[Schultz Thesis 2016]](https://doi.org/10.18453/rosdok_id00001765)

$D_{PW,WFS}(\mathbf{x}_0,\omega) = w(\mathbf{x}_0) \sqrt{\frac{\mathrm{j \omega}}{c}} \sqrt{8 \pi} \cdot
\sqrt{|\mathbf{x}_{Ref}-\mathbf{x}_{0}|} \cdot
\langle \mathbf{\hat{k}}_{PW}, \mathbf{\hat{n}}(\mathbf{x}_0) \rangle \cdot
\mathrm{e}^{- \mathrm{j} \frac{\omega}{c}\langle \mathbf{\hat{k}}_{PW}, \mathbf{x}_0 \rangle}.$

With
- $\mathbf{\hat{k}}_{PW} = (\cos\phi_{PW}, \sin\phi_{PW},0)^\text{T}$
- $\mathbf{x}_0=(r_0 \cos\phi_0, r_0 \sin\phi_0)^\text{T}$
- $\mathbf{\hat{n}}(\mathbf{x}_0)=-(\cos\phi_0, \sin\phi_0)^\text{T}$
- $\mathbf{x}_{Ref} = (0,0,0)^\text{T}$ and thus $|\mathbf{x}_{Ref}-\mathbf{x}_{0}|=|\mathbf{x}_{0}|=r_0$

we simplify and rearrange the dot products

$D_{PW,WFS}(\mathbf{x}_0,\omega) = -\sqrt{\frac{\mathrm{j \omega}}{c}} \sqrt{8 \pi r_0}
\cdot w(\mathbf{x}_0)
\cdot \cos(\phi_0-\phi_{PW})
\cdot \mathrm{e}^{- \mathrm{j} \frac{\omega}{c} r_0
\cos(\phi_0-\phi_{PW})}
$

We want to derive the Fourier series of the $\phi_0$-dependent terms in the above driving function. Once this is known, we can compare it with the Fourier series coefficients $D_{PW,HOA}$, that are inherently given in the 2.5D NFC-HOA plane wave driving function, stated here again
\begin{align}
D_{PW,HOA}(\mathbf{x}_0,\omega) = \sum\limits_{m=-\infty}^{+\infty} \underbrace{\frac{2 \mathrm{j}}{\frac{\omega}{c} r_0} \frac{(-\mathrm{j})^{|m|}}{h_{|m|}^{(2)}(\frac{\omega}{c} r_0)}
\mathrm{e}^{-\mathrm{j} m \phi_{PW}}}_{D_{PW,HOA}(m,\omega)}
\mathrm{e}^{\mathrm{j} m \phi_0}.
\end{align}

The Fourier analysis of the plane wave part is derived as, cf. [detailed calculus](https://github.com/spatialaudio/sfs-with-local-wavenumber-vector-concept/blob/master/nfc_hoa-vs-wfs_CylCoord.tex), also cf. [Nara Hahn AES140th, Table 1](http://www.aes.org/e-lib/browse.cfm?elib=18294)

$
\frac{1}{2 \pi}
\int\limits_0^{2 \pi}
\cos(\phi_0-\phi_{PW})
\, \mathrm{e}^{- \mathrm{j} \frac{\omega}{c} r_0
\cos(\phi_0-\phi_{PW})} \cdot \mathrm{e}^{- \mathrm{j} m \phi_0 } \mathrm{d} \phi_0 
= \frac{\mathrm{e}^{- \mathrm{j} m \phi_{PW}}}{2\,\mathrm{j}^{m-1}}
(J_{m-1}(\frac{\omega}{c} r_0) - J_{m+1}(\frac{\omega}{c} r_0))
$

denoting the $n$-th order [cylindrical Bessel function of first kind](https://dlmf.nist.gov/10.2) with $J_n(\cdot)$

The Fourier series of the spatial window $w(\mathbf{x}_0)$
\begin{equation}
w(\mathbf{x}_0) =
    \begin{cases}
      1 & \text{if}\qquad
       \mathbf{n}(\mathbf{x}_0) \cdot \hat{\mathbf{k}}_{PW}(\mathbf{x}_0) \geq 0 \\
      0 & \text{otherwise}
    \end{cases}.
\end{equation}
can be written as
\begin{align}
\frac{1}{2 \pi}
\int\limits_0^{2 \pi} w(\mathbf{x}_0) \cdot \mathrm{e}^{- \mathrm{j} m \phi_0 } \mathrm{d} \phi_0 = \frac{1}{2 \pi} \frac{-\mathrm{j}}{m} (\mathrm{e}^{- \mathrm{j} m (\phi_{PW}+\pi/2))}-\mathrm{e}^{- \mathrm{j} m (\phi_{PW}+3\pi/2)) }),
\end{align}
and is generally complex-valued.

For the special cases i) $\phi_{PW} = 0$, ii) $\phi_{PW} = \pi$  we obtain the real-valued Fourier series i) $-\frac{1}{2} \frac{\sin(m\frac{\pi}{2})}{m\frac{\pi}{2}}$, ii) $+\frac{1}{2} \frac{\sin(m\frac{\pi}{2})}{m\frac{\pi}{2}}$, with sinc()-funtion characteristics evaluated for discrete $m$.

The complex-valued convolution of both Fourier series together with the prefactors, i.e. the final Fourier series result, is thus 
\begin{align}
D_{PW,WFS}(m,\omega) = 
-\sqrt{\frac{\mathrm{j \omega}}{c}} \sqrt{8 \pi r_0} \cdot
\left[\frac{\mathrm{e}^{- \mathrm{j} m \phi_{PW}}}{2\,\mathrm{j}^{m-1}}
(J_{m-1}(\frac{\omega}{c} r_0) - J_{m+1}(\frac{\omega}{c} r_0))\right]
*_m
\left[\frac{1}{2 \pi} \frac{-\mathrm{j}}{m} (\mathrm{e}^{- \mathrm{i} m (\phi_{PW}+\pi/2))}-\mathrm{e}^{- \mathrm{j} m (\phi_{PW}+3\pi/2)) })\right]
\end{align}
doubtful to find an analytic expression for the result.

#### Acknowledgment
We thank [Naha Hahn](https://github.com/narahahn) for fruitful discussions and helpful comments on the calculus.

#### Copyright
This notebook is provided as [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use the notebook for your own purposes. The text is licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), the code of the IPython examples under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work with the github repository url [https://github.com/spatialaudio/sfs-with-local-wavenumber-vector-concept](https://github.com/spatialaudio/sfs-with-local-wavenumber-vector-concept).